# Download cHL Dataset and Preprocessing

This tutorial will walk you to prepare examples data from the cHL dataset (Shaban et al., MAPS) to use KRONOS. 

### Marker Metadata

Before running KRONOS, we need to define marker metadata. Unlike RGB images, where channels are fixed (red, green, blue) and normalization values are often hardcoded, spatial proteomics (SP) datasets vary in the number and type of channels. Marker names also differ in formatting (e.g., “KI67” might appear as “Ki-67” or “KI-67”) To address this, KRONOS expects a CSV file containing metadata for all markers embedded in the inference data. An initial CSV with 175 markers is provided in [marker_metadata.csv](https://huggingface.co/MahmoodLab/KRONOS/blob/main/marker_metadata.csv).

The marker metadata CSV includes four columns:
- **marker_name**: The name of the marker in uppercase.
- **marker_id**: A unique identifier assigned to the marker in the pretraining dataset.
- **marker_mean**: The mean intensity value of the marker, calculated from a reference dataset (e.g., KRONOS pretraining dataset).
- **marker_std**: The standard deviation intensity value of the marker, also calculated from a reference dataset.

<br/>

### How Marker IDs are assigned to Markers
Marker IDs are assigned as integers from 1 to 512. In the pretrained dataset, nuclear markers are assigned IDs from 1 to 127, while non-nuclear markers receive IDs from 128 to 512. This grouping helps capture high-level similarities between markers of the same type. Within each category, markers are arranged alphabetically, but only even-numbered IDs are assigned to those included in the pretrained dataset. The odd-numbered IDs are intentionally left unassigned, reserved for biologically similar markers that were not part of the pretrained dataset. This approach allows end-users to assign marker IDs from the odd-numbered values, ensuring that any newly added markers remain closely linked to the existing structure while preserving biological relevance.

## Step 1: Download dataset and marker metadata

In [2]:
# Download cHL dataset (MAPS) and marker metadata 
from huggingface_hub import hf_hub_download
import shutil
import os
from utils.chl_dataset_prep import download_and_prepare_chl_maps_dataset

# Set project dir 
project_dir = "./chl_maps_dataset"

# Download and prepare cHL dataset
download_and_prepare_chl_maps_dataset(project_dir)

# Download marker_metadata.csv 
cached_file = hf_hub_download(
    repo_id="MahmoodLab/KRONOS",
    filename="marker_metadata.csv"
)
shutil.copy(cached_file, os.path.join(os.path.join(project_dir, "dataset"), "marker_metadata.csv"))


marker_metadata.csv:   0%|          | 0.00/8.49k [00:00<?, ?B/s]

'./crc_maps_dataset/dataset/marker_metadata.csv'

## Step 2: Matching pretraining markers with cHL data markers 

The following script maps the marker information (stored in `marker_info.csv`) from the original dataset to those used in the KRONOS pretraining dataset (`marker_metadata.csv`). <br />
It also displays a list of unmatched markers along with suggestions derived from marker name similarity with entries in `marker_metadata.csv`.

In [3]:
from utils import MarkerMetadata
# Define the project directory
project_dir = "./chl_maps_dataset"  # Replace with your actual project directory
# Define paths for the dataset-specific marker info and the pretrained marker metadata files.
marker_info_csv_path = f"{project_dir}/dataset/marker_info.csv"        # Path to the dataset-specific marker info file.
marker_metadata_csv_path = f"{project_dir}/dataset/marker_metadata.csv"  # Path to the pretrained marker metadata file.
top_suggestions = 5  # Number of top suggestions to display for each unmatched marker.

# Create an instance of MarkerMetadata and retrieve the marker metadata.
obj = MarkerMetadata(marker_info_csv_path, marker_metadata_csv_path, top_suggestions)
obj.get_marker_metadata()

# Display the number of markers that do not match the pretrained dataset.
print(f"There are {len(obj.missing_marker_dict)} markers that do not match with the markers in the pretrained dataset.")

# Show the top suggestions based on marker name similarity for each unmatched marker.
print(f"Below are the top {top_suggestions} marker name similarity suggestions for each missing marker:")
display(obj.missing_marker_df)

# Display the dictionary for missing markers, which needs to be manually mapped to a biologically similar marker in marker_metadata.csv.
print("The following dictionary contains missing markers that need to be manually mapped:")
display(obj.missing_marker_dict)

There are 18 markers that do not match with the markers in the pretrained dataset.
Below are the top 5 marker name similarity suggestions for each missing marker:


,Suggestion 1,Suggestion 2,Suggestion 3,Suggestion 4,Suggestion 5
Missing Marker,,,,,
BCL-2,BCL2,BDCA-2,BCL6,B2M,CD28
CD46,CD61,CD54,CD45,CD40,CD36
COLLAGEN 4,COLLAGEN,LANGERIN,CTLA4,CATHEPSIN L,LAG3
CYTOKERITIN,CYTOKERATIN,CLUSTERIN,E-CADHERIN,CATHEPSIN L,LANGERIN
DAPI-01,DAPI,VDAC1,IL-1B,BDCA-2,PD1
GRANZYME B,GZMB,LYSOZYME,RELB,RB,LANGERIN
HLA-DR,HLA_DR,HLA_DRA,HLA_DRBPB,HLA1,HLA_1
IDO-1,IDO1,IL-1B,ARID1A,PD1,IGD
LAG-3,LAG3,SIGELC-3,LANGERIN,COLLAGEN,RELA


The following dictionary contains missing markers that need to be manually mapped:


{'BCL-2': '',
 'CD46': '',
 'COLLAGEN 4': '',
 'CYTOKERITIN': '',
 'DAPI-01': '',
 'GRANZYME B': '',
 'HLA-DR': '',
 'IDO-1': '',
 'LAG-3': '',
 'MMP-9': '',
 'MUC-1': '',
 'PD-1': '',
 'PD-L1': '',
 'T-BET': '',
 'TCR-G-D': '',
 'TCRB': '',
 'TIM-3': '',
 'VISA': ''}

## Step 3: Manual marker mapping 

If some markers do not match based on their names, you can manually adjust the mapping. Use the provided suggestions and/or the list of marker names in the marker_metadata.csv file. <br/>
Simply copy the dictionary syntax from the previous step and update the values for the unmatched markers with a valid, biologically similar marker from the suggestions or the `marker_metadata.csv` file.

In [4]:
obj.missing_marker_dict = {
    'BCL-2': 'BCL2',
    'CD46': '',
    'COLLAGEN 4': 'COLLAGEN',
    'CYTOKERITIN': 'CYTOKERATIN',
    'DAPI-01': 'DAPI',
    'GRANZYME B': 'GZMB',
    'IDO-1': 'IDO1',
    'LAG-3': 'LAG3',
    'MMP-9': 'MMP9',
    'MUC-1': 'MUC1',
    'PD-1': 'PD1',
    'PD-L1': 'PDL1',
    'T-BET': 'TBET',
    'TCR-G-D': 'TCR-GD',
    'TCRB': 'TCR-B',
    'TIM-3': 'TIM3',
    'VISA': ''
    }

# Retrieve marker metadata using the updated mapping.
obj.get_marker_metadata_with_mapping()

if len(obj.missing_marker_dict) > 0:
    # Display the count of markers that still do not match the pretrained dataset.
    print(f"There are {len(obj.missing_marker_dict)} markers that still do not match the markers in the pretrained dataset.")

    # Display the dataframe of unmatched markers.
    display(obj.missing_marker_df)

    # Display the dictionary of unmatched markers that require manual mapping.
    display(obj.missing_marker_dict)
else:
    print("All markers have been successfully mapped to the pretrained dataset.")

Marker TCR-G-D not found in metadata
Marker TCRB not found in metadata
There are 4 markers that still do not match the markers in the pretrained dataset.


,Suggestion 1,Suggestion 2,Suggestion 3,Suggestion 4,Suggestion 5
Missing Marker,,,,,
CD46,CD61,CD54,CD45,CD40,CD36
HLA-DR,HLA_DR,HLA_DRA,HLA_DRBPB,HLA1,HLA_1
TCR-G-D,TCR_GD,TCR_B,DC-SIGN,IGD,CD3
TCRB,TCR_B,RB,TCR_GD,ERB,TCF1
VISA,VISTA,INOS,IGA2,IGA1,ICOS


{'CD46': '', 'TCR-G-D': 'TCR-GD', 'TCRB': 'TCR-B', 'VISA': ''}

## Step 4 (Optional): Manually Set Metadata
If some markers are still unmatched with the pretrained dataset and you can not ignore these marker then you can manually assign their marker ID, mean, and standard deviation values:

- **Marker ID**: Choose an unassigned ID from the range 1–512 in marker_metadata.csv. Ideally, select an ID close to a biologically similar marker.
- **Mean & Std Values**: Calculate these from your dataset for the corresponding markers. Ensure marker intensities are converted to float type and intensities are in range of 0-1 before computing the mean and standard deviation.

In [6]:
marker_metadata_dict = {
        'CD46': {'marker_id': 295, 'marker_mean': 0.051, 'marker_std': 0.085},
        'VISA': {'marker_id': 45, 'marker_mean': 0.015, 'marker_std': 0.014},
    }

obj.set_marker_metadata(marker_metadata_dict)
# Display the count of markers that still do not match the pretrained dataset.
if len(obj.missing_marker_dict) > 0:
    print(f"There are {len(obj.missing_marker_dict)} markers that still do not match the markers in the pretrained dataset.")
    
    # Display the dataframe of unmatched markers.
    display(obj.missing_marker_df)

    # Display the dictionary of unmatched markers that require manual mapping.
    display(obj.missing_marker_dict)
else:
    print("All markers now have valid metadata.")
display(obj.marker_info)

There are 2 markers that still do not match the markers in the pretrained dataset.


,Suggestion 1,Suggestion 2,Suggestion 3,Suggestion 4,Suggestion 5
Missing Marker,,,,,
HLA-DR,HLA_DR,HLA_DRA,HLA_DRBPB,HLA1,HLA_1
TCR-G-D,TCR_GD,TCR_B,DC-SIGN,IGD,CD3
TCRB,TCR_B,RB,TCR_GD,ERB,TCF1


{'TCR-G-D': 'TCR-GD', 'TCRB': 'TCR-B'}

,channel_id,marker_name,marker_id,marker_mean,marker_std
0,0,BCL-2,150,0.047104,0.060276
1,1,CCR6,166,0.044867,0.042833
2,2,CD11B,180,0.032169,0.052366
3,3,CD11C,182,0.019039,0.044336
4,4,CD15,194,0.016322,0.040416
5,5,CD16,196,0.041869,0.055626
6,6,CD162,198,0.012217,0.040094
7,7,CD163,200,0.014384,0.033087
8,8,CD2,212,0.161256,0.110404
9,9,CD20,214,0.045192,0.057727


## Step 5: Save Final Dataset Specific Metadata File

In [7]:
output_csv_path = f"{project_dir}/dataset/marker_info_with_metadata.csv"
obj.export_marker_metadata(output_csv_path)
display(obj.marker_info)

Exported marker metadata to ./crc_maps_dataset/dataset/marker_info_with_metadata.csv


,channel_id,marker_name,marker_id,marker_mean,marker_std
0,0,BCL-2,150,0.047104,0.060276
1,1,CCR6,166,0.044867,0.042833
2,2,CD11B,180,0.032169,0.052366
3,3,CD11C,182,0.019039,0.044336
4,4,CD15,194,0.016322,0.040416
5,5,CD16,196,0.041869,0.055626
6,6,CD162,198,0.012217,0.040094
7,7,CD163,200,0.014384,0.033087
8,8,CD2,212,0.161256,0.110404
9,9,CD20,214,0.045192,0.057727
